# Тетрадь 2: Преобразователи данных

Предметом изучения в данной тетради являются преобразователи данных. Мы разберём, что они 
из себя представляют, как они устроены и для чего нужны.

## Содержание

- [Теория](#Теория)
- [Примеры](#Примеры)
- [Задания](#Задания)

***

## Теория

### Преобразование и предварительная обработка данных

**Предварительная обработка данных** (Data Preprocessing) в машинном обучении — это набор методов
и техник, которые применяются к исходным данным перед их использованием для обучения модели. Цель 
предварительной обработки — подготовить данные к анализу, улучшить их качество и сделать их более 
пригодными для обучения моделей машинного обучения.

Вообще, существует множество различных этапов предварительной обработки данных. Однако в контексте 
данного курса мы будем относить всякие манипуляции, будь то очистка, сокращение размерности или 
форматирование, к преобразованиям данных. Отсюда **преобразователи данных** — программные интерфейсы 
(в основном классы), которые переводят данные из одного состояние в другое.

Проиллюстрируем такой этап предобработки, как дополнение данных. Суть дополнения в том, что 
информация, которую вы где-либо раздобыли для анализа далеко не всегда является полной (например, в
таблице присутствуют пустые поля). Ниже представлена диаграмма, которая схематично описывает эффект
применения так называемого `Preprocessor`'а к «сырым» данным.

<div style="text-align: center">
    <img src="images/imputer_scheme_plot.png" alt="Imputer Scheme">
</div>

### Некоторые этапы предварительной обработки данных

Возвращась к разнообразию этапов предварительной обработки данных, пречислим некоторые из них: 

- **Разделение:** тренировочный/тестовый/валидационный набор; кросс-валидация.

- **Очистка:** удаление дубликатов; обработка пропущенных значений; удаление шума.

- **Масштабирование:** нормализация/Стандартизация (например, Min-Max нормализация, 
  Z-масштабирование); логарифмирование/экспоненцирование.

- **Кодирование:** бинарное кодирование (One-Hot Encoding); лэйбл-кодирование (Label Encoding).

- **Уменьшение Размерности:** PCA (Principal Component Analysis); t-SNE (t-distributed Stochastic 
  Neighbor Embedding); LDA (Linear Discriminant Analysis).

### Порядок выполнения этапов предварительной обработки

Очень важно помнить, что порядок выполнения этих этапов действительно имеет огромное значение.
Сначала выполняется разделение данных, затем очистка, а потом уже всё остальное. Также далее мы 
обсудим не менее значимый нюанс касаемо применения преобразователей к отдельным наборам данных.

***

## Примеры

Рассмотрим теперь несколько упрощённых и один полный пример преобразователей данных. Начнём с 
более простых функций, а далее продемонстрируем полноценный класс. В качетве преобразования для 
примера возьмём Z-масштабирование. Формула для этой операции выглядит следующим образом:

$$Z(X) = \frac{X - \textrm{mean}(X)}{\textrm{std}(X)},$$

где:
- $X$ — множество всех точек данных признака;
- $\textrm{mean}(X)$ — среднее по $X$;
- $\textrm{std}(X)$ — стандартное отклонение по $X$.

Запрограммируем функцию для Z-масштабирования.

In [2]:
import numpy as np
from numpy import ndarray


def z_scale(data: ndarray) -> ndarray:
    scaled = (data - np.mean(data)) / np.std(data)
    return scaled

Теперь представим некоторый случайный набор данных и применим к нему нашу функцию.

In [ ]:
(size, round_to) = ((10, 4), 3)
selection = np.round(np.random.standard_normal(size), round_to)
scaled = np.round(z_scale(selection), round_to)

print(f"Before scaling:\n{selection}\n\nAfter scaling:\n{scaled}")

Как видим, всё работает отлично! Однако, есть нюанс, о котором было упомянуто в конце
теоретического раздела, и сейчас мы его обусдим.

Итак, представьте, что вы обучаете модель на тренировочных данных. Вы готовите её к взаимодействию 
с информацией, которую она ещё не видела. Но во время преобразования данных вы применяете
инструмент трансформации к каждому из наборов (тренировочному и тестовому) по отдельности.

То есть та же функция `z_scale()` принимает во внимание характеристики тестового набора данных при
его масштабировании. Теперь представим, что тестового набора нет. Вам только предстоит его получить
и обработать. Получается, для преобразования тестового набора данных нам нужно заглянуть в будущее!

Многие начинающие часто совершают эту ошибку. Новички трансформируют тестовые данные, пользуясь их 
характеристиками. Хоть это и явное противоречие, часто его никто не замечает. Так что же делать с 
этим противоречием?

Для решения этой проблемы нам понадобится хранить параметры тренировочных данных (минимум,
максимум, среднее, медиану, стандартное отклонение и т.п.) и использовать их для преобразований
тестовых. Рассмотрим конкретную реализацию.

### Базовый класс преобразователей

Для начала определим интерфейс преобразователя в удобной в пользовании форме. За основу возьмём те
же классы-трансформеры из библиотеки Scikit-Learn.

In [27]:
from abc import ABC, abstractmethod
from typing import Any, Tuple

from numpy import ndarray


class BasePreprocessor(ABC):
    # The Base Preprocessor class is an abstract base class for preprocessor
    # implementations.

    def __init__(self, copy: bool = True) -> None:
        self.copy = copy

    @abstractmethod
    def fit(self, x: ndarray) -> None:
        # Fit the preprocessor to the provided features.
        message = "Every preprocessor must implement the `fit()` method."
        raise NotImplementedError(message)

    @abstractmethod
    def transform(self, x: ndarray) -> Any:
        # Transform the input features.
        message = "Every preprocessor must implement the `transform()` method."
        raise NotImplementedError(message)

    @staticmethod
    def _get_values_masks(array: ndarray) -> Tuple[bool, bool]:
        non_zero_values_mask = array != 0
        zero_values_mask = ~non_zero_values_mask
        return non_zero_values_mask, zero_values_mask

Как видим, это [абстрактный базовый класс](https://docs.python.org/3/library/abc.html). При 
инициализации мы задаём параметр `copy`, который нам пригодится в дальнейшем. Главное — у класса 
`BasePreprocessor` есть методы `fit()` и `transform()`. Первый отвечает за хранение характеристик 
данных, а второй за применение транформации с учётом самых характеристик. Есть также защищённый 
метод `_get_values_masks()`. О нём мы тоже поговорим чуть позже.

### Z-масштабировщик

Рассмотрим теперь конкретную реализаицю Z-масштабирующего преобразователя.

In [5]:
class ZScalingPreprocessor(BasePreprocessor):
    # Z-Scaling Preprocessor inherits from Base Preprocessor to provide method
    # signatures and calls to `_get_values_masks()` through self-ref.

    def __init__(self, copy: bool = True) -> None:
        super().__init__(copy)
        self.means: ndarray
        self.stds: ndarray

    def fit(self, x: ndarray) -> None:
        # This method memorizes the data parameters of the input as attributes
        # to use this information for Z-Scale calculation.
        self.means = np.nanmean(x, axis=0)
        self.stds = np.nanstd(x, axis=0)

    def transform(self, x: ndarray) -> ndarray:
        # This method applies the scaling formula excluding zero elements from
        # the process.

        # Use the copy if `copy` parameter was specifyed
        if self.copy:
            x = x.copy()

        # Get zero and nonzero elements positions to avoid artefacts
        (nonzero_std_mask, zero_std_mask) = self._get_values_masks(self.stds)
        (nonzero_mean_mask, _) = self._get_values_masks(self.means)
        x[:, zero_std_mask] = 0

        # Use the Z-Scale formula
        x[:, nonzero_std_mask] = (
            x[:, nonzero_std_mask] - self.means[nonzero_mean_mask]
        ) / self.stds[nonzero_std_mask]
        return x

    def fit_transform(self, x) -> ndarray:
        # Fit and transform at the same time."""

        self.fit(x)

        transformed = self.transform(x)
        return transformed

Проясним некоорые моменты более подробно:

- При вычислении среднего и стандартного отклочения в методе `fit()`, мы используем функции с 
  приставкой `nan*`. Это значит, что при подсчётах ячейки, содержащие `NaN` или любые другие 
  недействительные или нечисловые значения, будут проигнорированы.
- Перед масштабированием с применением формулы мы отсекаем точки данных, что представляют собой 
  нули, для того чтобы избежать артефактов в виде отрицательных значений.

Теперь посмотрим, насколько удачно можно применить вышеописанный интерфейс.

In [ ]:
from modules.preprocessing import DataSplitter


splitter = DataSplitter(permute=True)
scaler = ZScalingPreprocessor(copy=True)

(x, y) = (selection[:, :-1], selection[:, -1].reshape((-1, 1)))
(x_train, x_test, y_train, y_test) = splitter.split_data(x, y, test_size=0.25)

nonscaled = (x_train, x_test, y_train, y_test)
names = ("X-train", "X-test", "y-train", "y-test")
for nscld, name in zip(nonscaled, names):
    print(f"{name}:\n{nscld}\n")

In [ ]:
scaled = (
    scaler.fit_transform(x_train),
    scaler.transform(x_test),
    scaler.fit_transform(y_train),
    scaler.transform(y_test),
)

names = ("X-train scaled", "X-test scaled", "y-train scaled", "y-test scaled")
for scld, name in zip(scaled, names):
    print(f"{name}:\n{np.round(scld, 3)}\n")

***

## Задания

### Задание 1

По аналогии с заданием в примерах реализуйте класс для MinMax масштабирования, наследуя от базового 
класса `BasePreprocessor`. В качестве подсказки рассмотрите следующую формулу:

$$ \textrm{MinMax(X)} = \frac{X - \textrm{min}(X)}{\textrm{max}(X) - \textrm{min}(X)}, $$

где:
- $X$ — множество всех точек данных признака;
- $\textrm{min}(X)$ — минимум по $X$;
- $\textrm{max}(X)$ — максимум по $X$.

В процессе решения не забудьте про маски для нулевых значений!

In [9]:
class MMScalingPreprocessor(BasePreprocessor):
    # MinMax Scaling Preprocessor inherits from Base Preprocessor to provide
    # method signatures and provide calls to `_get_values_masks()` through
    # self-ref.

    def __init__(self, copy: bool = True) -> None:
        super.__init__(copy)
        self.min_values: ndarray
        self.max_values: ndarray

    def fit(self, x: ndarray) -> None:
        # This method should fit the transformer to the data.
        ...

    def transform(self, x: ndarray) -> Any:
        # This method should transform the data accordingly to the state,
        # produced by the :meth:`fit()` method.
        ...

    def fit_transform(self, x: ndarray) -> Any:
        # This method just combines both :meth:`fit()` and :meth:`transform()`.
        ...

Когда реализуете класс из ячейки выше, запускайте ячейку ниже и продемонстрируйте правильность 
выполнения задания.

In [ ]:
splitter = DataSplitter(permute=True)
scaler = MMScalingPreprocessor(copy=True)

(x, y) = (selection[:, :-1], selection[:, -1].reshape((-1, 1)))
(x_train, x_test, y_train, y_test) = splitter.split_data(
    x, y, test_size=0.25, random_seed=2024
)

nonscaled = (x_train, x_test, y_train, y_test)
names = ("X-train", "X-test", "y-train", "y-test")
for nscld, name in zip(nonscaled, names):
    print(f"{name}:\n{nscld}\n")

### Задание 2*

Данное задание обладает повышенной сложностью выполнения. Тут вам предстоит самостоятельно и без
каких-либо подсказок реализовать класс `ImputingPreprocessor`, который будет иметь функционал,
описанный в [теории](#теория). Данный интерфейс должен быть способен выполнить замену недостающих
значений (например, `NaN`) по трём стратегиям: константа, среднее и медиана по столбцу. **Важно
помнить, что все дополнения воспроизводятся относительно столбцов!**

In [25]:
class ImputingPreprocessor(BasePreprocessor):
    # This class should implement an interface for imputing the missing values
    # by one of these strategies: "constant", "mean" and "median".
    ...

Так же, как и в случае с предыдущим заданием, выполните ячейку ниже, когда закончите выполнять
задание.

In [ ]:
from numpy.random import random as nprand


na_shape = (5, 5)
nan_array = np.random.standard_normal(na_shape)
for i in range(na_shape[0]):
    for j in range(na_shape[1]):
        nan_array[i, j] = np.nan if nprand() > 0.8 else nan_array[i, j]

imputer = ImputingPreprocessor(...)
na_imputed = imputer.fit_transform(nan_array)

print(f"Before imputing:\n{nan_array}\n")
print(f"After imputing:\n{na_imputed}\n")

***

## Выводы

В настоящей тетради мы: поближе познакомились с предварительной обработкой данных и некоторыми
её этапами; изучили, что такое преобразователи данных, для чего они нужны, как ими пользоваться,
а самое главное — научились самостоятельно их воспроизводить!

Если есть интерес, можете попробовать далее модифицровать код из секций выше. Также можете 
попробовать написать собственный интерфейс, который вам покажется более удобным.